In [1]:
from typing import Optional, Type
from langchain.tools.base import BaseTool
from typing import List
from pydantic import BaseModel, Field, ConfigDict
import json
from square.client import Client
from square.api.orders_api import OrdersApi

In [2]:
from dotenv import load_dotenv

# load in keys from .env. Have in format API_KEY="".
load_dotenv()

True

In [3]:
import os
API_KEY = os.getenv("API_KEY")
LOCATION = os.getenv("LOCATION")
GOOGLE_PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID") # ie, confident-jackle-123456

In [4]:
# square device id presets. More info at https://developer.squareup.com/docs/devtools/sandbox/testing
DEVICE_ID_CHECKOUT_SUCCESS="9fa747a2-25ff-48ee-b078-04381f7c828f"
DEVICE_ID_CHECKOUT_SUCCESS_TIP="22cd266c-6246-4c06-9983-67f0c26346b0"
DEVICE_ID_CHECKOUT_SUCCESS_GC="4mp4e78c-88ed-4d55-a269-8008dfe14e9"
DEVICE_ID_CHECKOUT_FAILURE_BUYER_CANCEL="841100b9-ee60-4537-9bcf-e30b2ba5e215"

In [5]:
client = Client(
    square_version='2023-08-16',
    access_token=API_KEY,
    environment='sandbox')

In [6]:
# some helper functions for easy api debugging
def handle_response(result):
    if result.is_success():
      print(result.body)
    elif result.is_error():
      print(result.errors)

# search for orders at a specified location.
def search_orders(location: str, return_entries: bool = False):
    body = { 'location_ids': [location], "return_entries": return_entries}
    result = client.orders.search_orders(body)
    handle_response(result)

# Create a terminal checkout - default is payment success.
def create_terminal_checkout(order_id: str, cost, device_id: str = DEVICE_ID_CHECKOUT_SUCCESS):
    result = client.terminal.create_terminal_checkout(
      body = {
        "idempotency_key": str(uuid4().hex),
        "checkout": {
          "amount_money": {
            "amount": cost,
            "currency": "USD"
          },
          "order_id": order_id,
          "payment_options": {
            "autocomplete": True,
            "accept_partial_authorization": False
          },
          "device_options": {
            "device_id": device_id,
            "skip_receipt_screen": True,
            "collect_signature": True,
            "show_itemized_cart": True
          },
          "payment_type": "CARD_PRESENT",
          "customer_id": ""
        }
      }
    )
    handle_response(result)

# get the status of a current terminal by checkout ID.
def get_terminal_checkout(checkout_id: str):
    result = client.terminal.get_terminal_checkout(checkout_id)
    handle_response(result)


In [7]:
from google.cloud import aiplatform

aiplatform.init(project=GOOGLE_PROJECT_ID, location="us-central1")

# if authentication doesn't work with just above, uncomment below.
# !gcloud auth application-default set-quota-project GOOGLE_PROJECT_ID

In [9]:
from langchain.tools import StructuredTool
from uuid import uuid4

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class SearchOrderTool(BaseTool):
    name="search_order_tool"
    description="for searching for orders at a location"
    def _run(
          self, location_id: str, run_manager: Optional[CallbackManagerForToolRun] = None
      ) -> str:
          """Use the tool."""
          return client.orders.search_orders(body = {"location_ids": [location_id], "return_entries": True}).body['order_entries']

In [49]:

class CreateOrderSchema(BaseModel):
    location_id: str = Field(description="Should be a valid location id provided in the prompt. ")
    name: str = Field(description="The name of the specified food item. If no food item is identified, choose a random one.")
    quantity: str = Field(description="**VERY IMPORTANT: must be a string. The quantity of food item requested.")
    amount: int = Field(description="**VERY IMPORTANT: absolutely must be an integer. The cost of the food item. ")
    currency: str = Field(description="The currency being paid with, defaults to 'USD'.")


class CreateOrderTool(BaseTool):
    name="create_new_order"
    description="used for creating a new order"
    args_schema: Type[CreateOrderSchema] = CreateOrderSchema
    def _run(
        self, location_id: str, name: str, quantity: str, amount: int, currency: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:    
          return client.orders.create_order(body = {  "order": { "location_id": location_id, "line_items": [{"name": name, "quantity": quantity, "modifiers": [], "base_price_money": { "amount": amount,"currency": currency}}]},"idempotency_key": str(uuid4().hex)})
          

In [50]:
from langchain.llms import vertexai
from vertexai.preview.language_models import TextGenerationModel
from langchain.chains import APIChain
from langchain.agents import AgentType, initialize_agent

#define the tools available to the agent
tools = [CreateOrderTool(), SearchOrderTool()]

# use PaLM 2 via vertex
llm = vertexai.VertexAI(temperature=0)

# initialize the agent - has to be STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION because CreateOrderTool takes multiple inputs.
agent = initialize_agent(tools,
                         llm, 
                         agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True
                        )


In [21]:
#agent.run("location_id = LETE3KSW0H1RB. I want to order a $7 coffee, 3 burgers for $12 and a tequila shot.")

# adding memory to the agent